In [1]:
import sys
sys.path.append('../../utils')
import numpy as np 
import matplotlib.pyplot as plt
import os
from scipy import ndimage
import skimage.measure as measure 
from tqdm import tqdm
from skimage import color
import seaborn as sns 
from natsort import natsorted
import re
import collections
import pandas as pd
from tqdm import tqdm

import tifffile as tf
from skimage.transform import downscale_local_mean

from aicsimageio.writers import OmeTiffWriter

In [2]:
'paths to directories'
data_root = '/home/kkobayas/koseki-data/Raman/data/unet/fovs/'
save_root = '/home/kkobayas/koseki-data/Raman/data/unet/fovs_bin4/'

# BF channel
bf_c = 0
# Nanog channel
# gene_c = 1
# gene_name = 'Krt7'
# gene channels
# genes = {'Nanog':1, 'Col1a1':2, 'Utf1':3, 'Krt7':1, 'Peg10':2, 'Bgn':3, 'Fabp1':1, 'Epcam':2, 'Nnat':3}
genes = {'Fabp7':1, 'Epcam':2, 'Nnat':3}

bin_size = 4

In [3]:
# D8_Pos6_Fabp7_Epcam_Nnat.ome.tiff
expression = "(?P<day_pos>D.*_Pos\d*)_.*.ome.tiff"

# i=0

for g in genes.keys():
    for _,_,files in os.walk(data_root):
        for file in tqdm(files):
    #         i = i+1
    #         if i<138:
    #             continue
    #         if (file=="D8.5_Pos19_Nanog_Col1a1_Utf1.ome.tiff") or (file=="D14.5_Pos4_Nanog_Col1a1_Utf1.ome.tiff") or (file=="D13.5_Pos10_Nanog_Col1a1_Utf1.ome.tiff") or (file=="D10.5_Pos0_Nanog_Col1a1_Utf1.ome.tiff"):

            if file.find(g) != -1:
#                 print(file)
                fp = os.path.join(data_root, file)
                img = tf.imread(fp)
                img_bin = downscale_local_mean(img,(1,1,bin_size,bin_size))

                p = re.compile(expression)

                m=p.match(file)
                if m != None:
                    day_pos = m.group('day_pos')

                    if not os.path.isdir(os.path.join(save_root, g)):
                        os.mkdir(os.path.join(save_root, g))
                    fn = day_pos+"_"+g+".ome.tiff"
                    save_f = os.path.join(save_root,g,fn)

                    OmeTiffWriter.save(img_bin[[bf_c, genes[g]],:,:,:], save_f, dim_order='CZYX')


100%|████████████████████████████████████████████████████████████████████████████| 882/882 [26:00<00:00,  1.77s/it]
